In [1]:
from mlx_whisper.audio import (
    FRAMES_PER_SECOND,
    HOP_LENGTH,
    N_FRAMES,
    N_SAMPLES,
    SAMPLE_RATE,
    log_mel_spectrogram,
    pad_or_trim,
)
from mlx_whisper.tokenizer import (
    LANGUAGES,
    get_tokenizer
)
from mlx_whisper.transcribe import (
    ModelHolder
)

/opt/miniconda3/envs/mlenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
audio = "./mlx_whisper/assets/ls_test.flac"
#audio = "/Users/dlewis/sample.wav"

In [3]:
import mlx.core as mx
dtype = mx.float16
model = ModelHolder.get_model('mlx-community/whisper-tiny', dtype=dtype)
print(model)

Fetching 4 files: 100%|██████████| 4/4 [00:00<00:00, 45466.71it/s]

Whisper(
  (encoder): AudioEncoder(
    (conv1): Conv1d(80, 384, kernel_size=3, stride=1, padding=1, dilation=1, bias=True)
    (conv2): Conv1d(384, 384, kernel_size=3, stride=2, padding=1, dilation=1, bias=True)
    (blocks.0): ResidualAttentionBlock(
      (attn): MultiHeadAttention(
        (query): Linear(input_dims=384, output_dims=384, bias=True)
        (key): Linear(input_dims=384, output_dims=384, bias=False)
        (value): Linear(input_dims=384, output_dims=384, bias=True)
        (out): Linear(input_dims=384, output_dims=384, bias=True)
      )
      (attn_ln): LayerNorm(384, eps=1e-05, affine=True)
      (mlp1): Linear(input_dims=384, output_dims=1536, bias=True)
      (mlp2): Linear(input_dims=1536, output_dims=384, bias=True)
      (mlp_ln): LayerNorm(384, eps=1e-05, affine=True)
    )
    (blocks.1): ResidualAttentionBlock(
      (attn): MultiHeadAttention(
        (query): Linear(input_dims=384, output_dims=384, bias=True)
        (key): Linear(input_dims=384, output_

In [4]:
print(model.dims)
print(model.num_languages)

ModelDimensions(n_mels=80, n_audio_ctx=1500, n_audio_state=384, n_audio_head=6, n_audio_layer=4, n_vocab=51865, n_text_ctx=448, n_text_state=384, n_text_head=6, n_text_layer=4)
99


In [5]:
mels = log_mel_spectrogram(audio, n_mels=model.dims.n_mels, padding=N_SAMPLES)
print(mels.shape)

(3666, 80)


In [6]:
print("N_FRAMES:", N_FRAMES)
content_frames = mels.shape[-2] - N_FRAMES
print("content_frames:", content_frames)


N_FRAMES: 3000
content_frames: 666


In [7]:
print("HOP_LENGTH:", HOP_LENGTH)
print("SAMPLE_RATE:", SAMPLE_RATE)
content_duration=float(content_frames * HOP_LENGTH / SAMPLE_RATE)
print("content_duration:", content_duration)

HOP_LENGTH: 160
SAMPLE_RATE: 16000
content_duration: 6.66


In [8]:
tokenizer = get_tokenizer(model.is_multilingual, num_languages=model.num_languages, language="en", task="transcribe")
print(tokenizer)

Tokenizer(encoding=<Encoding 'multilingual.tiktoken'>, num_languages=99, language='en', task='transcribe', sot_sequence=(50258, 50259, 50359), special_tokens={'<|15.74|>': 51151, '<|18.04|>': 51266, '<|26.32|>': 51680, '<|24.82|>': 51605, '<|6.84|>': 50706, '<|lo|>': 50336, '<|19.92|>': 51360, '<|10.72|>': 50900, '<|17.44|>': 51236, '<|4.04|>': 50566, '<|29.14|>': 51821, '<|0.60|>': 50394, '<|9.82|>': 50855, '<|26.16|>': 51672, '<|2.12|>': 50470, '<|1.94|>': 50461, '<|10.10|>': 50869, '<|0.10|>': 50369, '<|2.00|>': 50464, '<|10.16|>': 50872, '<|26.74|>': 51701, '<|17.12|>': 51220, '<|25.54|>': 51641, '<|10.14|>': 50871, '<|15.42|>': 51135, '<|15.50|>': 51139, '<|16.10|>': 51169, '<|19.52|>': 51340, '<|24.78|>': 51603, '<|29.24|>': 51826, '<|et|>': 50307, '<|17.04|>': 51216, '<|12.66|>': 50997, '<|23.34|>': 51531, '<|23.24|>': 51526, '<|7.32|>': 50730, '<|24.70|>': 51599, '<|17.46|>': 51237, '<|18.36|>': 51282, '<|13.78|>': 51053, '<|24.26|>': 51577, '<|22.46|>': 51487, '<|13.74|>': 510

In [9]:
mels.shape

(3666, 80)

In [10]:
print(pad_or_trim(mels, N_FRAMES, axis=-2).shape)

(3000, 80)


In [13]:
decoded_output = model.decode(pad_or_trim(mels, N_FRAMES, axis=-2).astype(dtype))
print("decoded_output", decoded_output)
print("audio_features.shape:", decoded_output.audio_features.shape)
print("# of tokens:", len(decoded_output.tokens))

x.shape after conv1 & gelu: (1, 3000, 384)
x.shape after conv2 & gelu: (1, 1500, 384)
decoder called
decoder called
decoder called
decoder called
decoder called
decoder called
decoder called
decoder called
decoder called
decoder called
decoder called
decoder called
decoder called
decoder called
decoder called
decoder called
decoder called
decoder called
decoder called
decoder called
decoder called
decoder called
decoder called
decoder called
decoder called
decoder called
decoder called
decoder called
decoder called
decoder called
decoded_output DecodingResult(audio_features=array([[0.0966187, 0.0881348, 0.0394287, ..., -0.0284424, 0.03302, 0.0341187],
       [-0.307861, 0.607422, 1.05566, ..., 1.43164, 0.203125, 1.11328],
       [0.104736, 2.32812, 1.78027, ..., 0.924805, -0.26001, 0.84668],
       ...,
       [0.703125, -1.76855, 0.348145, ..., -0.218506, -0.447266, 0.517578],
       [-0.0490723, -0.612793, 0.148315, ..., 0.677246, 0.078186, 0.381104],
       [0.20459, -0.0919189, -1.

In [12]:
print(mels.shape)
print(mx.expand_dims(pad_or_trim(mels, N_FRAMES, axis=-2), 0).shape)
print(model.encoder(mx.expand_dims(pad_or_trim(mels, N_FRAMES, axis=-2), 0)).shape)

(3666, 80)
(1, 3000, 80)
x.shape after conv1 & gelu: (1, 3000, 384)
x.shape after conv2 & gelu: (1, 1500, 384)
(1, 1500, 384)
